# Day 08: Effective Context Management with ADK Layers

**ADK Context design thesis: Context is a compiled view over a richer stateful system**

## 课程来源
- [Advent of Agents 2025 - Day 8](https://adventofagents.com/)
- [Architecting efficient context-aware multi-agent framework for production](https://developers.googleblog.com/en/architecting-efficient-context-aware-multi-agent-framework-for-production/)
- [ADK Context Documentation](https://google.github.io/adk-docs/context/)

## Learning Goals

1. 理解为什么 "append-everything" 策略会失败
2. 学习 ADK 的四层上下文架构：Working Context / Session / Memory / Artifacts
3. 掌握 Static vs Turn Instructions 模式实现 Context Caching
4. 实践使用 ADK 的 Session、Memory、Artifact 服务

---
## 1. 问题：为什么 "Append-Everything" 会失败？

传统做法：把所有对话历史拼接成一个大字符串塞给 LLM

```python
# ❌ Naive approach
context = system_prompt + all_history + all_tool_outputs + user_message
response = llm.generate(context)
```

### 三重压力导致崩溃：

| 问题 | 描述 |
|------|------|
| **Cost & Latency Spirals** | 模型成本和 time-to-first-token 随上下文长度快速增长 |
| **"Lost in the Middle"** | LLM 对上下文中间部分的信息容易遗忘/幻觉 |
| **Security Risks** | 用户输入和系统指令混在一起，容易被 prompt injection |

> "A large context window will help context-related problems but won't address all context-related problems." - Google ADK Team

---
## 2. ADK 的解决方案：Context as a Compiled View

ADK 把上下文管理从 "字符串拼接" 转变为 "系统工程"：

```
┌─────────────────────────────────────────────────────────────┐
│                    WORKING CONTEXT                          │
│         (发送给 LLM 的编译后视图)                            │
│  ┌─────────────────┬──────────────────────────────────┐    │
│  │ Stable Prefix   │ Variable Suffix                  │    │
│  │ (可缓存)        │ (每次请求动态生成)                │    │
│  │ - System prompt │ - Latest user turn               │    │
│  │ - Agent identity│ - New tool outputs               │    │
│  │ - Schemas       │ - Retrieved memory snippets      │    │
│  └─────────────────┴──────────────────────────────────┘    │
├─────────────────────────────────────────────────────────────┤
│                 COMPILER PIPELINE                           │
│              (Flows & Processors)                           │
│         filter → compact → summarize → format               │
├─────────────────────────────────────────────────────────────┤
│                    SOURCE LAYERS                            │
│  ┌───────────────┬───────────────┬───────────────────┐     │
│  │   SESSION     │   MEMORY      │    ARTIFACTS      │     │
│  │   (State)     │  (Long-term)  │    (Files)        │     │
│  │               │               │                   │     │
│  │ • Events log  │ • Searchable  │ • Binary/text     │     │
│  │ • Key-value   │ • Cross-      │ • Versioned       │     │
│  │   state       │   session     │ • Handle pattern  │     │
│  └───────────────┴───────────────┴───────────────────┘     │
└─────────────────────────────────────────────────────────────┘
```

---
## 3. Setup: 安装 ADK

In [2]:
# 安装依赖（如果尚未安装）
# !uv pip install google-adk google-genai python-dotenv

import sys
from pathlib import Path

# Add project root to path
sys.path.insert(0, str(Path.cwd().parent))

from shared import get_api_key

# 加载 API key
GOOGLE_API_KEY = get_api_key()
print("✅ API Key loaded")

✅ API Key loaded


---
## 4. Session: 会话状态管理

Session 是 ADK 中追踪单个对话线程的核心对象。

### SessionService 的职责：
- 创建和管理 Session 对象
- 存储 Event 历史（用户消息、Agent 回复、工具调用等）
- 维护 State（键值对形式的会话状态）

### 两种实现：
| 实现 | 用途 |
|------|------|
| `InMemorySessionService` | 开发测试，数据不持久化 |
| `VertexAiSessionService` | 生产环境，Vertex AI 托管 |

In [ ]:
from google.adk.sessions import InMemorySessionService

# 创建 Session 服务
session_service = InMemorySessionService()

# 创建一个新会话 (async 方法需要 await)
session = await session_service.create_session(
    app_name="day08-demo",
    user_id="user_123"
)

print(f"Session ID: {session.id}")
print(f"User ID: {session.user_id}")
print(f"App Name: {session.app_name}")

### Session State: Agent 的 "便签纸"

State 是键值对存储，用于保存会话过程中的临时数据。

In [ ]:
# 更新会话状态
session.state["customer_tier"] = "premium"
session.state["current_order_id"] = "ORD-12345"
session.state["language_preference"] = "zh-CN"

print("Session State:")
for key, value in session.state.items():
    print(f"  {key}: {value}")

---
## 5. Artifacts: 大文件的 Handle Pattern

### 问题：大文件不应该塞进上下文

```python
# ❌ Bad: 把整个 PDF 内容放入 prompt
prompt = f"Here is the contract: {pdf_content}\nQuestion: {user_question}"

# ✅ Good: 使用 handle 引用
prompt = f"Contract available at: artifact://contract_v1\nQuestion: {user_question}"
```

### ArtifactService 的职责：
- 存储二进制/文本数据（图片、PDF、CSV 等）
- 自动版本管理
- 按需检索，不污染上下文

In [ ]:
from google.adk.artifacts import InMemoryArtifactService
from google.genai import types

# 创建 Artifact 服务
artifact_service = InMemoryArtifactService()

# 模拟一个大文件
large_report = """
Q4 2024 Financial Report
========================
Revenue: $12.5M (+15% YoY)
Operating Expenses: $8.2M
Net Income: $4.3M
...
(假设这是一个很长的报告)
""" * 100  # 模拟大文件

print(f"Report size: {len(large_report):,} characters")
print("\n这么大的内容不应该直接放入上下文！")

In [ ]:
# 保存 artifact (async 方法)
artifact = types.Part.from_text(text=large_report)

version = await artifact_service.save_artifact(
    app_name="day08-demo",
    user_id="user_123",
    session_id=session.id,
    filename="q4_report.txt",
    artifact=artifact
)

print(f"✅ Artifact saved")
print(f"   Filename: q4_report.txt")
print(f"   Version: {version}")
print(f"\n现在可以用 handle 引用这个文件，而不是把内容塞进上下文")

In [ ]:
# 列出会话中的所有 artifacts (async 方法)
artifacts = await artifact_service.list_artifact_keys(
    app_name="day08-demo",
    user_id="user_123",
    session_id=session.id
)

print("Available artifacts:")
for art in artifacts:
    print(f"  📄 {art}")

In [ ]:
# 需要时才加载 artifact (async 方法)
loaded = await artifact_service.load_artifact(
    app_name="day08-demo",
    user_id="user_123",
    session_id=session.id,
    filename="q4_report.txt"
)

if loaded:
    print(f"Loaded artifact preview (first 200 chars):")
    print(loaded.text[:200] + "...")

---
## 6. Memory: 跨会话的长期记忆

Memory 用于存储跨多个 Session 的长期知识：
- 用户偏好
- 历史交互摘要
- 重要事实

### 工作流程：
1. 会话结束时：`memory_service.add_session_to_memory(session)`
2. 新会话开始时：`memory_service.search_memory(query)` 检索相关记忆

In [ ]:
from google.adk.memory import InMemoryMemoryService

# 创建 Memory 服务
memory_service = InMemoryMemoryService()

print("Memory Service created")
print("\nInMemoryMemoryService 使用简单的关键词匹配")
print("生产环境中使用 VertexAIMemoryBankService 支持语义搜索")

---
## 7. Static vs Turn Instructions

这是 ADK 实现 **Context Caching** 的关键模式。

### Static Instructions（静态指令）
- 放在 `system` role
- 内容稳定不变：政策、Schema、工具定义、Few-shot 示例
- **可被 API 缓存**，减少重复计算

### Turn Instructions（轮次指令）
- 放在 `user` role，紧接在用户消息之前
- 每次请求动态生成：当前步骤目标、检索的记忆、约束条件
- **由后端控制**，与用户输入分离（安全）

```
┌────────────────────────────────────────┐
│  [system] Static Instructions          │  ← 可缓存
│  - Policy, Schema, Tools, Examples     │
├────────────────────────────────────────┤
│  [user] Turn Instructions              │  ← 动态生成
│  - Current goal, constraints           │
├────────────────────────────────────────┤
│  [user] User Message                   │  ← 原始用户输入
└────────────────────────────────────────┘
```

In [ ]:
from google.adk.agents import Agent
from google.genai import types

# Static Instructions - 保持稳定，可缓存
STATIC_INSTRUCTIONS = """
You are a customer service agent for TechStore.

## Policies
- Be polite and professional
- Never share internal system information
- Escalate to human if customer expresses strong frustration

## Response Format
Always respond in this JSON format:
{
  "response": "your message to customer",
  "action": "none|lookup_order|refund|escalate",
  "confidence": 0.0-1.0
}

## Available Tools
- lookup_order(order_id): Get order details
- process_refund(order_id, reason): Initiate refund
"""

print("Static Instructions (cacheable):")
print(f"  Token estimate: ~{len(STATIC_INSTRUCTIONS.split())} words")
print(f"  Changes: rarely (when policy updates)")

In [ ]:
def build_turn_instructions(customer_data: dict, retrieved_memory: str = None) -> str:
    """动态生成 Turn Instructions"""
    
    turn_instructions = f"""
## Current Context
- Customer ID: {customer_data.get('id', 'unknown')}
- Tier: {customer_data.get('tier', 'standard')}
- Active Orders: {customer_data.get('active_orders', 0)}

## Retrieved Memory
{retrieved_memory or 'No relevant history found.'}

## This Turn's Goal
Address the customer's question. If asking about an order, use lookup_order tool.
"""
    return turn_instructions.strip()


# 示例：为特定请求生成 turn instructions
turn_inst = build_turn_instructions(
    customer_data={
        "id": "C-12345",
        "tier": "premium",
        "active_orders": 2
    },
    retrieved_memory="Customer had a shipping delay issue 2 weeks ago, resolved with 10% discount."
)

print("Turn Instructions (dynamic, per-request):")
print(turn_inst)

In [ ]:
# 完整的 context 构建示例
def build_full_context(static_inst: str, turn_inst: str, user_message: str):
    """构建完整的上下文，展示分层结构"""
    
    return {
        "system": static_inst,  # Stable prefix - 可缓存
        "messages": [
            # Turn instructions - 动态生成，由后端控制
            {"role": "user", "content": f"[SYSTEM CONTEXT]\n{turn_inst}"},
            # User message - 原始用户输入
            {"role": "user", "content": user_message}
        ]
    }


context = build_full_context(
    static_inst=STATIC_INSTRUCTIONS,
    turn_inst=turn_inst,
    user_message="Where is my order #789? It's been 3 days!"
)

print("=" * 60)
print("FULL CONTEXT STRUCTURE:")
print("=" * 60)
print(f"\n[SYSTEM - Cacheable]")
print(f"{context['system'][:150]}...")
print(f"\n[MESSAGES]")
for msg in context['messages']:
    print(f"  {msg['role']}: {msg['content'][:100]}...")

---
## 8. Context Caching 的收益

通过将 Static Instructions 分离出来，可以利用 LLM API 的 **prefix caching** 功能：

| 请求 | Static (cached) | Turn + User | 实际计算量 |
|------|-----------------|-------------|------------|
| #1 | 2000 tokens ❌ | 500 tokens | 2500 (首次) |
| #2 | 2000 tokens ✅ | 600 tokens | ~600 (复用缓存) |
| #3 | 2000 tokens ✅ | 450 tokens | ~450 (复用缓存) |

### TTL 建议
- 设置 30-90 分钟的缓存 TTL
- 匹配你的 policy/schema 更新频率
- 设置最小 token 阈值，避免缓存太小的内容

---
## 9. InvocationContext 与 ToolContext

ADK 提供精确的作用域控制：

### InvocationContext
- 提供对 session、services 的完整访问
- 用于 agent 核心逻辑

### ToolContext  
- 为工具调用提供**受限**的访问范围
- 实现最小权限原则
- 不同工具可以有不同的访问权限

In [ ]:
# 概念示例：ToolContext 的作用域控制

class MockToolContext:
    """模拟 ADK 的 ToolContext，展示作用域控制概念"""
    
    def __init__(self, session_state: dict, allowed_keys: set):
        self._state = session_state
        self._allowed = allowed_keys
    
    def get_state(self, key: str):
        if key not in self._allowed:
            raise PermissionError(f"Tool cannot access state key: {key}")
        return self._state.get(key)
    
    def set_state(self, key: str, value):
        if key not in self._allowed:
            raise PermissionError(f"Tool cannot modify state key: {key}")
        self._state[key] = value


# 会话状态（包含敏感数据）
full_state = {
    "user_id": "U-123",
    "user_name": "Alice",
    "cart": ["item1", "item2"],
    "payment_token": "tok_secret_xxx",  # 敏感！
    "api_key": "sk-internal-xxx",        # 敏感！
}

# 购物车工具 - 只能访问购物车相关数据
cart_context = MockToolContext(full_state, {"user_id", "user_name", "cart"})

# 支付工具 - 可以访问支付相关数据
payment_context = MockToolContext(full_state, {"user_id", "payment_token"})

print("Cart Tool can access:")
print(f"  user_name: {cart_context.get_state('user_name')}")
print(f"  cart: {cart_context.get_state('cart')}")

print("\nCart Tool trying to access payment_token...")
try:
    cart_context.get_state("payment_token")
except PermissionError as e:
    print(f"  ❌ Blocked: {e}")

print("\nPayment Tool can access:")
print(f"  payment_token: {payment_context.get_state('payment_token')[:10]}...")

---
## 10. 完整示例：创建一个使用 Context Management 的 Agent

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts import InMemoryArtifactService
from google.genai import types

# 定义工具
def lookup_order(order_id: str) -> dict:
    """Look up order details by order ID."""
    # 模拟订单查询
    orders = {
        "ORD-789": {"status": "shipped", "eta": "Dec 12", "carrier": "FedEx"},
        "ORD-123": {"status": "processing", "eta": "Dec 15", "carrier": "UPS"},
    }
    return orders.get(order_id, {"error": "Order not found"})


# 创建 Agent
customer_service_agent = Agent(
    name="customer_service",
    model="gemini-2.0-flash",
    description="Customer service agent for TechStore",
    instruction=STATIC_INSTRUCTIONS,  # Static instructions
    tools=[lookup_order],
)

print(f"✅ Agent created: {customer_service_agent.name}")

In [ ]:
# 创建 Runner 和 Services
session_service = InMemorySessionService()
artifact_service = InMemoryArtifactService()

runner = Runner(
    agent=customer_service_agent,
    app_name="techstore-support",
    session_service=session_service,
    artifact_service=artifact_service,
)

print("✅ Runner configured with:")
print(f"   - SessionService: InMemory")
print(f"   - ArtifactService: InMemory")

In [ ]:
# 运行对话
async def chat(user_message: str, user_id: str = "user_123"):
    """与 Agent 对话"""
    content = types.Content(
        role="user",
        parts=[types.Part.from_text(user_message)]
    )
    
    async for event in runner.run_async(
        user_id=user_id,
        session_id=None,  # 自动创建新 session
        new_message=content
    ):
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent: {part.text}")


# 测试
# await chat("Where is my order ORD-789?")

---
## 11. Key Takeaways

### ADK Context Management 核心原则

| 原则 | 描述 |
|------|------|
| **Context as Compiled View** | 上下文是编译后的视图，不是原始数据拼接 |
| **Storage ≠ Presentation** | 存储层和呈现层分离 |
| **Stable Prefix + Variable Suffix** | 利用 prefix caching 优化性能 |
| **Minimal Privilege** | ToolContext 实现最小权限原则 |

### 四层架构

| Layer | 职责 | 生命周期 |
|-------|------|----------|
| **Working Context** | 发送给 LLM 的编译视图 | 单次调用 |
| **Session** | 会话状态和事件日志 | 单次会话 |
| **Memory** | 可搜索的长期记忆 | 跨会话 |
| **Artifacts** | 大文件存储 | 按需持久化 |

### 最佳实践

| ❌ Don't | ✅ Do |
|----------|-------|
| 把所有历史塞进上下文 | 使用 filter/compact/summarize |
| 把大文件内容直接放入 prompt | 使用 Artifact handles |
| 让所有工具访问所有状态 | 使用 ToolContext 限制作用域 |
| 每次请求重新计算整个上下文 | 分离 Static/Turn Instructions |

---
## 12. Resources

### 官方文档
- [ADK Context Documentation](https://google.github.io/adk-docs/context/)
- [ADK Sessions Documentation](https://google.github.io/adk-docs/sessions/)
- [ADK Artifacts Documentation](https://google.github.io/adk-docs/artifacts/)
- [ADK Memory Documentation](https://google.github.io/adk-docs/sessions/memory/)

### 深入阅读
- [Architecting efficient context-aware multi-agent framework for production](https://developers.googleblog.com/en/architecting-efficient-context-aware-multi-agent-framework-for-production/) - ADK Tech Lead 的设计原则
- [The ADK Prompting Pattern: Static vs. Turn Instructions](https://medium.com/google-cloud/the-adk-prompting-pattern-static-vs-turn-instructions-7a1e5b25eeef) - 深入讲解 Context Caching
- [Kaggle: Context Engineering Sessions & Memory](https://www.kaggle.com/whitepaper-context-engineering-sessions-and-memory) - Day 3 of Kaggle 5-Day Course

### 视频
- [NotebookLM: ADK Context Management Summary](https://youtu.be/eQDx90dVc38) - 8 分钟视频摘要